In [1]:
import networkx as nx
import urllib.request
import json
import pandas as pd
import re
import pickle
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

### Now that we cleaned up the data we can build the network.

In [2]:
data_path = 'data/'
data_path = 'data/'
file = 'links2022.ndjson'

data = pd.read_json(data_path + file, lines=True)
data = data.rename(columns={'out_going':'author','in_going':'adressee'})
author_posts = dict()
for post in tqdm(range(data.shape[0])):
    # Create a dictionary of authors and their flair, who they answered and their comment
    author_name = data['author'][post]
    author_flair = data['flair'][post]
    answered = data['adressee'][post]
    comment = data['comment'][post]
    comment_id = data['id'][post]
    
    # Add author to dictionary or add attributes
    if author_name not in author_posts:
        author_posts[author_name] = [author_flair,[answered],comment,[comment_id]]
    else:
        author_posts[author_name][1].append(answered)
        author_posts[author_name][2] += comment
        author_posts[author_name][3].append(comment_id)

/var/folders/xr/mtd0_wg95rg8n506mvct3vfm0000gn/T/ipykernel_72873/3800550492.py:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(data_path + file, lines=True)


ValueError: Expected object or value

### Build Graph

In [ ]:
# Create a directed graph
DirectedGraph = nx.MultiDiGraph()
for author_name in author_posts.keys():
    # Add nodes to the graph
    DirectedGraph.add_node(author_name,flair=author_posts[author_name][0],comments=author_posts[author_name][2],comment_id=author_posts[author_name][3])

for author_name in author_posts.keys():
    # Add edges
    for edge_to in author_posts[author_name][1]:
        if edge_to != None:
            DirectedGraph.add_edge(author_name,edge_to)

# Create folder if it does not exist
if not os.path.exists('LargeData'):
    os.makedirs('LargeData')
    
# Save authors as a pickle file
pickle.dump(DirectedGraph, open('LargeData/Directed2022.pkl','wb'))

### Save data

In [ ]:
pickle.dump(data, open('LargeData/data2022.pkl','wb'))